# PROJEKTNA NALOGA IZ ANALIZE PODATKOV IGRE LEAGUE OF LEGENDS

## UVOD

[League of Legends](https://www.leagueoflegends.com/en-gb/) je priljubljena videoigra, ki je izšla že leta 2009. Od takrat je močno zrasla v popularnosti, danes jo namreč igra okoli 150 milijonov igralcev. Podatke za analizo te igre sem vzel iz spletne strani [u.gg](https://u.gg/).

Osnovni način igre, ki sem ga izbral za analizo podatkov, se imenuje *Summoner's Rift*. V tem načinu se spopadeta dve ekipi, vsaka po 5 igralcev. Vsak igralec mora izmed 168 likov (*champion-ov*) izbrati svojega. Vsak igralec ekipe dobi tudi svojo [vlogo](https://www.pinnacle.com/en/esports-hub/betting-articles/league-of-legends/the-different-league-of-legends-roles-explained/atz2jajnpca9ppgg) (*role*), to so: top, jungle, mid, bot in support. Ekipa mora s pomočjo vseh igralcev in strategije uničiti nasprotnikovo bazo, da zmaga igro. Igralci so rangirani v 15 nivojev glede na to, kako dobro igrajo. Kjer ni posebej specificirano sem analiziral podatke iz vseh nivojev skupaj.

## ANALIZA PODATKOV

### SPLOŠNI PODATKI

Za uživalca te priljubljene igre je najbolj zanimiv podatek, kateri championi najpogosteje zmagujejo (*win rate*), kateri so najpogosteje izbrani (*pick rate*) in, kateri so najpogosteje prepovedani za igranje s strani soigralcev (*ban rate*). 

In [ ]:
import pandas as pd
import matplotlib as plt

